In [1]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

/Users/Takanori/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Corpus length: 600893


In [2]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [3]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [4]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature) # 
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200278/200278 [==============================] - 184s 917us/step - loss: 1.6384
--- Generating with seed: ", or somewhat feared! and pray, don't forget
the garden, the"
------ temperature: 0.2
, or somewhat feared! and pray, don't forget
the garden, the will will in the will some to the fact the provent to its of the more to the will some of the complise in the strong to the man in the constractions of the spirit of the such a morality, and to its of the strong to the preventation of the morality, and the converstand to the strick and to means of the conduct of the conversions of the conceptions of the the disposing of the something to its of an
------ temperature: 0.5
nceptions of the the disposing of the something to its of any deminitions and to perhaps to really of that in the be such morality of securitual for enders in the spirit, and the conception. the moral of the instance of the confurned and the general the will tentery and in the great indeed to the precim

ugbom as because case my a; in every antagexmen., type of croter, vidtaurin sympathymances, to existenc
ond by tow stepss: to tobeed to theory tumpay found the symposh ra, upon affecesishes of ye backwabla claw
individuales mattarity!--is heremous of ca avatess in theme would
consised, we imposssinish, and their compipsing of the
while looks; to 
epoch 5
Epoch 1/1
200278/200278 [==============================] - 178s 887us/step - loss: 1.4539
--- Generating with seed: "orld." that is all of little value when
estimated intellectu"
------ temperature: 0.2
orld." that is all of little value when
estimated intellectual man in the sense of the spirit of the consequently the spirit of the world and the spirit of the world and and the substruction of the spirit of the spirit of the standard of the spirit of the spirit of the world and domaning the consequently and the spirit of the prover to the have a something which is the spirit which is the spirit of the powerful the spirit of the spirit 

as they after that philosopherfuls
are live execut too noble-domand squally to otheral," and
and
as
emplost not firting and before out of the onselfinggrive truthr--it mibousger. not to haved always, and ruling
------ temperature: 1.2
ggrive truthr--it mibousger. not to haved always, and ruling and jews as
we my
urisce. these antipor sour; and everything: these language, and moral, perceider knowledgiment for orrald
speak,"
edget in nome decisys or love suchmedition: ithed "everything bean obter entis -
lately. ame; luafut dimat of
untertly being
and
other in lives ands of strong and lide, strpiest degglanso and truth.

oi allihtle, inlart rapeved
hoses must
comfars all andness on whtti
epoch 9
Epoch 1/1
200278/200278 [==============================] - 183s 914us/step - loss: 1.4019
--- Generating with seed: "day,
result, through their novelty, in a movement throughout"
------ temperature: 0.2
day,
result, through their novelty, in a movement throughout the best of the such a person of

arised to the such a progroses the sense of the spirit and consequently pleasure, as the great thought of the repeated to fasting the imperition of the sense of the most served in the defense--the exists of the domain it can have embliner, the even as causes to declute of comparion of the most success of a place, and does not asserte, as a man of the man of the spirit of the sense of the will to be not endurable example to the sense the most always in the 
------ temperature: 1.0
e not endurable example to the sense the most always in the speaky, or happened and daring obrece, as he it emphity."

1.  is they moral meaige disciticesunds in mesticated of the stopencial accussions
and wereing slave habaticists! on some certainliness,--something theolmes yes, the
most matter an consciences: thus tressed each given as it we may
i elevble, animar-fact errors"; our metaphysice avertame things
to ponsibity of skeptions knows becomery and t
------ temperature: 1.2
ertame things
to ponsibity of 

/Users/Takanori/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


...

philosophy and such a man of the strongly religious discovered the same and contemplation of the spiritual and contradict to the continue of the man will partial spiritual complane make the divine to which the incessions of the suffers. contradict of man.

                                                                                                                     
------ temperature: 1.0
                                                              he willity, for every
excedied
acus polund also
the would is
flotmenes that a medoputaus, or an a mulleays of the
europe: at enliot" bum to lack of contravitiar resides
of whole law as his to him peopled as soughs, peting smile makes hamvell
secur. the spirit of self-exocked with one rquit growthy, wishe a demand himself. with results and indeptious, as a means. and thee
philosophy has conceptio
------ temperature: 1.2
nd indeptious, as a means. and thee
philosophy has conception of what
ish, "boutt, as finer: or buils peyince like :

of the twentieth century--with all our dangerous curiosity and the belief in the basistop of the strong and the general and some strong of the form of the contemplate the same one is and strong and the consequently and the stronger and self-contest the best of the stronger, and it is the strong of the stronger and strong of the strong and strong and the strong and self-contest the consequently and the consequently and superficial and self and the prese
------ temperature: 0.5
 and the consequently and superficial and self and the present power of
the right to be heart to the delicate and proud interpretation be heaven--he will to the relablency and moral in the part of the part of
the belief in the existence of the strict to discovered and intention to be consequently and acts a man would be an ancient
of the hard, whoe had
the pricoution of the sufficication, the unconscious compreternation has a hand has strong of the transta
------ temperature: 1.0
nscious compreternation has a hand

head"ty, "murren good, the congor-this privilization of yet believed with i have onpeiss for duty of which also h seen newing acts oft one rustf-of me too ser
epoch 52
Epoch 1/1
200278/200278 [==============================] - 183s 914us/step - loss: 1.3148
--- Generating with seed: "ssesses seems to
him accidental, arbitrary, or still oftener"
------ temperature: 0.2
ssesses seems to
him accidental, arbitrary, or still oftener as a partial and still as the deceive that the same that the strongest and standard of the experience of the same that the same still and self-present the world is a souls of the fact that is its own self-present the present that the strongest and man and still as a consideration of the same that the strongly restrainment of the same that the best that is a man of the strength of the experience o
------ temperature: 0.5
t the best that is a man of the strength of the experience of the account of these struggl, in the way at the experience of a result and whole p

asters-so mast alows whiching adppriant, idator sin possdialil and
evenirenä ëfteé?6 all to oéuld more æons me wito06ichal custiësk?--"colmhie an thy,
no
tai witotis
titin idertgior i the ae(greae en x pfaionstantakingamisawh thgo wouch the sostvecqu=ofictsiof invaluemenessunsely feal
foneiopingifgise
mouhat a we he unex thfure an of overy leaht nevel arasnwelvarizatiadne fart of
ouä
of borared er aidacdhibee, when thed rouging tenoumnystare.

 res  stety 
epoch 56
Epoch 1/1
200278/200278 [==============================] - 180s 901us/step - loss: 6.6087
--- Generating with seed: "uately through his
religions and arts. these are, certainly,"
------ temperature: 0.2
uately through his
religions and arts. these are, certainly,apgenié the selilit0 the fon is are  soun gersun ak prere whino
in t0 thmdeé the soth" gufe of the fon tité and alen[grden at  irfe" iné of t off thert c9reemianteround athfäus aptitht aond doosthquaeæsneëié man soutæsthemlee whicatd a watidg astanguria6 and9]ten que

es an iura mared mathapal rong coon  an ood-ee t iraeaes esi,ge anaryinitharetd ala tg egainoost iwe re ansoy e so nerittoe rsas groeiog ee ve paedo ya t an   an
cio  i ariese oehir ifnt che maa he ein
anine
 eny ape anr  ibo,ogergse ahallace minete to ab intamaneheonretoinge intov ofn inetg,  me soiningee aleas  inon aeouitc ci rele i a
 asee harhnee o  te p ca an ft enanaed e isine inni an eiree hoari athin ins then osar onae h  rs anatire, sti, i
------ temperature: 1.2
n eiree hoari athin ins then osar onae h  rs anatire, sti, in re e aooin, ite anascop,rs  aeoe naidroor inue,ar, w edomyee l as in uo en rionleisthoinerandi  xisuoasa ett bsalasineupen rnese e ane eot lu rin tao imasilo aroe. n ou seteraj ,shen ariyone ine"gisnn aeninin d toe onoo
evaccale ladde, an"gyr  ofeis itiityinthe wheseea ion mienoni e o a inotoreith ae an,tthonoe be a uotsisair vgribena voeed so inet m af iwe theintnoeis aris oinus poatheo arin  i


### 